##실행 방법

- 해당 파일(노트북)의 셀을 위에서부터 순서대로 실행하십시오.

- 프로젝트 폴더 경로를 루트(root) 경로로 설정하면 정상 동작합니다.

## 모델 사용 안내

- 파인튜닝된 모델 가중치가 Models 폴더에 포함되어 있습니다.

- 만약 직접 모델을 학습한 후 사용하고 싶다면, **Model_Training.ipynb**에서 학습을 진행한 뒤 해당 모델을 로드해 활용할 수 있습니다.

In [ ]:
!pip install -q accelerate
!pip install -q peft

!pip install -q transformers
!pip install -q autoawq
!pip install -q bitsandbytes
!pip install -q trl

!pip install -q langchain
!pip install -q langchain_community
!pip install -q langchain_openai

!pip install -q sentence_transformers

!pip install -q chromadb
!pip install -q faiss-cpu
!pip install -q langchainhub
!pip install -q rank_bm25

!pip install -q pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               1.5.2
aiohappyeyeballs                         2.6.1
aiohttp                                  3.11.14
aiosignal                                1.3.2
alabaster                                1.0.0
albucore                                 0.0.23
albumentations                           2.0.5
ale-py                                   0.10.2
altair                                   5.5.0
annotated-types                          0.7.0
anyio                                    4.9.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
array_record                             0.7.1
arviz                                    0.21.0
asgiref                                  3.8.1
astropy                                  7.0.1
astropy-iers-data                    

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/HansolDeco_S3_AI_Construction_Safety')

Mounted at /content/drive


In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import gc
import numpy as np

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.runnables import RunnableMap, RunnableLambda, RunnableSequence, RunnablePassthrough

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import bitsandbytes as bnb
import torch
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
import random
from rank_bm25 import BM25Okapi

from sentence_transformers import SentenceTransformer

import umap
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from Utils.data_preprocessing import data_preprocessing
from Utils.Retriever import retriever, retriever_runnable, ready_retriever, post_retriever
from Utils.prompts import retriever_prompt, non_retriever_prompt, refine_prompt
from Evaluate.Matrix import cosine_similarity_sbert, jaccard_similarity, final_score

In [ ]:
train = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')
combined_training_data, combined_test_data = data_preprocessing(train, test)
validation_data = combined_training_data.sample(n=200, random_state=42)
train_data = combined_training_data.drop(validation_data.index).reset_index(drop=True)
validation_data = validation_data.reset_index(drop=True)

In [ ]:
ready_retriever("intfloat/multilingual-e5-large-instruct", train_data)

Loading model: intfloat/multilingual-e5-large-instruct


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

...

Cause embeddings computed.
Computing Environment embeddings for train_data...


Batches:   0%|          | 0/726 [00:00<?, ?it/s]

Environment embeddings computed.
Computing Answer embeddings for train_data...


Batches:   0%|          | 0/726 [00:00<?, ?it/s]

Answer embeddings computed.
Computing BM25 corpus...
BM25 corpus for accident_cause and accident_environment prepared.


In [ ]:
embedding_model_name = "jhgan/ko-sbert-sts"
embedding_model = SentenceTransformer(embedding_model_name)
save_directory = "./Models/DAPT_Model(gemma2)"
model = AutoModelForCausalLM.from_pretrained(save_directory, trust_remote_code=True, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(save_directory, trust_remote_code=True)


text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
<ipython-input-7-48b30d4e0c1a>:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
post_model_id = 'kakaocorp/kanana-nano-2.1b-instruct'

post_model = AutoModelForCausalLM.from_pretrained(post_model_id ,trust_remote_code=True, device_map={"": 0})
post_tokenizer = AutoTokenizer.from_pretrained(post_model_id, trust_remote_code=True)


post_generation_pipeline = pipeline(
    model=post_model,
    tokenizer=post_tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

post_llm = HuggingFacePipeline(pipeline=post_generation_pipeline)

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
generate_input = RunnableMap({
    'retrieved_documents' : retriever_runnable,
    'accident_info' : lambda inputs: inputs["accident_info"]
})

retriever_chain = generate_input | retriever_prompt | llm.bind(stop=["\n\n"])

In [ ]:
# retriever 함수를 체인에 사용할 수 있도록 래핑하는 함수
def post_retriever_runnable(inputs):
    """
    입력 딕셔너리(inputs)는 다음 키들을 포함해야 합니다.
      - "accident_cause": 사고 원인 텍스트
      - "accident_environment": 사고 환경 텍스트
      - "accident_info": 기타 사고 정보 (프롬프트에 그대로 전달)

    반환값은 "retrieved_documents"를 포함하는 딕셔너리입니다.
    """
    answer = inputs.get("answer", "")
    # retriever() 함수 호출 (여기서는 top_k=5, alpha=0.8, beta=0.2 예시)
    retrieved_docs, scores = post_retriever(answer)
    return retrieved_docs


In [ ]:
post_generate_input = RunnableMap({
    'retrieved_documents' : post_retriever_runnable,
    'answer' : lambda inputs: inputs["answer"],
    'question' : lambda inputs: inputs["question"]

})
post_process = RunnableLambda(
    lambda text: text.split('\n\n')[0].strip().replace("재발방지대책 및 향후조치계획: ", "")
)

In [ ]:
retriever_chain = generate_input | retriever_prompt | llm.bind(stop=["\n\n"])

post_chain = post_generate_input | refine_prompt | post_llm.bind(stop=["\n\n"]) | post_process



In [ ]:
post_text=[]

for idx in tqdm(range(len(combined_test_data))):

    input_data1 = {
    "accident_cause": combined_test_data.loc[idx, 'accident_cause'],
    "accident_environment": combined_test_data.loc[idx, 'accident_environment'],
    "accident_info": combined_test_data.loc[idx, 'question']}
    answer = retriever_chain.invoke(input_data1)
    input_data2 = {'answer' : answer,
              'question' : combined_test_data.loc[idx, 'accident_cause']}
    post_text.append(post_chain.invoke(input_data2).strip())

  0%|          | 0/964 [00:00<?, ?it/s]

In [ ]:
post_text

['아웃트리거 바닥 고임목을 3단으로 보강하는 등의 조치를 취하였으나 지반침하가 발생한 사실을 고려하여 아웃트리거 받침목 설치 시 지반의 지내력을 사전에 확인하고, 고정 상태를 철저히 점',
 '작업자 안전교육 실시 및 보안면 착용 의무화, 숫돌 측면 사용 금지 교육 실시.',
 '작업 전 안전교육 실시와 작업 시 주변을 확인하고 이동하는 등의 안전작업 지시.',
 '작업발판 위 자재 정리 철저 및 작업자 안전교육 실시.',
 '안전난간 설치 및 점심시간 작업 지양을 통한 재발 방지 대책.',
 '작업발판 설치와 작업자 안전교육 강화를 통한 재발 방지 대책 및 향후 조치 계획.',
 '매설작업 시 신호수 배치를 통한 작업 추진과 안전 교육 실시.',
 '작업 전 TBM을 통한 작업순서 및 안전작업 방법 교육 실시, 작업자 안전교육 및 TBM을 통한 안전수칙 준수, 작업자 개인보호구 착용 철저, 작업 시작 전 작업순서와 방법 철저 숙지, 안전수칙 준수',
 '작업자 안전교육 및 안전관리 철저에 대한 재발 방지 대책과 향후 조치 계획.',
 '작업 전 TBM을 통한 작업순서 및 안전작업절차 교육 실시, 해체작업 시 안전대 걸이시설 설치 및 착용 확인, 작업자 안전교육 실시, 해체작업 시 충돌방지 조치 및 작업자 보호 조치 강화.',
 '고령 작업자 및 건강상태 불량자에 대한 현장 작업 금지 지시.',
 '장비 이동 시 철판 궤도 설치 및 관리 철저\n이동 경로 사전 점검 및 안전성 확보\n이동 작업 전 작업자 안전 교육 실시',
 '자재 운반 시 2인 1조 작업 및 작업 전 안전교육 실시, 자재 정리 및 작업공간 확보, 작업자 안전교육 및 안전장구 착용 철저.',
 '관리 직원에 대한 안전교육 실시와 현장 관리감독 강화를 통한 재발 방지 대책. 작업자 안전교육 실시와 현장 관리감독 강화를 통한 재발 방지 대책.',
 '작업 전 안전교육 철저와 현장 내 안전관리 철저를 통한 재발 방지 대책.',
 '작업자 안전관리교육 실시와 개구부 위험작업 시 안전난간 설치.',


In [ ]:
combined_test_data['answer'] = post_text

In [ ]:
# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding_model.encode(post_text)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [ ]:
submission = pd.read_csv('./Data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = post_text
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./submission_final_v1.csv', index=False, encoding='utf-8-sig')

In [ ]:
submission

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,아웃트리거 바닥 고임목을 3단으로 보강하는 등의 조치를 취하였으나 지반침하가 발생한...,-0.382703,-0.190014,0.242829,1.186753,-0.565788,0.227122,0.955873,-0.207931,...,0.055061,0.551877,-0.182066,0.897394,0.844159,1.182262,0.076669,-0.495599,0.624733,0.060529
1,TEST_001,"작업자 안전교육 실시 및 보안면 착용 의무화, 숫돌 측면 사용 금지 교육 실시.",-0.259239,0.565507,-0.715998,0.154411,-0.706902,0.196600,1.274439,0.558698,...,0.478528,0.625284,0.522889,1.311739,1.033384,-0.332220,0.026709,-0.494137,0.162767,-0.485754
2,TEST_002,작업 전 안전교육 실시와 작업 시 주변을 확인하고 이동하는 등의 안전작업 지시.,-0.662292,0.184229,0.132109,0.884519,-0.226394,1.012324,-0.374678,-0.441849,...,-0.038788,1.964092,0.518754,1.546506,1.544919,-0.450633,-0.933618,0.284504,0.426995,0.355141
3,TEST_003,작업발판 위 자재 정리 철저 및 작업자 안전교육 실시.,0.609163,0.855517,-0.593585,0.529149,-0.139028,0.937730,0.739635,0.045973,...,0.242368,1.118531,0.121235,1.340071,0.636126,-0.245145,0.253370,-0.631849,0.610805,0.278057
4,TEST_004,안전난간 설치 및 점심시간 작업 지양을 통한 재발 방지 대책.,-1.128903,0.388636,-0.556258,1.326017,-0.002680,0.381639,1.063460,0.692995,...,-0.163933,0.941222,0.106837,2.597677,0.016096,-0.347819,-0.068200,0.333404,1.272658,0.151798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,TEST_959,"석재의 품질을 확인하고, 품질이 불량한 경우 사용을 금지하는 조치. 석재의 품질을 ...",0.268186,0.129812,-0.167861,0.740362,0.629550,-0.130685,-0.164606,0.021404,...,1.216733,0.295471,0.143586,0.215256,0.970788,-0.158281,0.850085,0.120648,0.553863,0.275392
960,TEST_960,"작업장 이동통로 확보와 작업 전 TBM 시 안전교육 강화, 사고 조치 후 재발 방지...",-0.720433,0.428550,-0.037009,0.012686,-0.024855,0.662496,0.353885,-0.415413,...,-0.045727,1.697502,0.290558,2.477941,1.332314,-0.463444,-0.394514,0.002466,1.330798,-0.275141
961,TEST_961,작업 전 안전교육 실시 및 믹서트럭 슈트 아래로 이동 시 안전한 이동 방법 교육 실시.,-0.803203,-0.798069,-0.215471,1.477507,-1.182041,0.727368,0.481827,0.016586,...,0.019679,1.070091,-0.167855,1.955203,1.616903,-0.297339,-0.636665,0.008249,0.806437,-0.004599
962,TEST_962,"깨진 타일은 밀폐용기에 담아 처리하고, 작업자는 보호장갑을 착용하여 작업하여야 한다.",0.404695,0.637496,-1.403666,0.696192,-1.017786,-0.429042,-0.153588,0.110024,...,0.198564,0.600845,-0.005314,-0.218798,0.693558,0.289856,-0.290527,-1.015513,0.099689,-0.240395
